In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [7]:
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(url)

df=df[0]

df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [25]:
df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df.head()




,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')
df.head()


,Postcode,Borough,Neighbourhood
0,M4P,Central Toronto,Davisville North
1,M9A,Queen's Park,Queen's Park
2,M3L,North York,Downsview West
3,M1V,Scarborough,"Steeles East,Milliken,L'Amoreaux East,Agincour..."
4,M3C,North York,"Don Mills South,Flemingdon Park"


In [10]:
df.shape

(103, 3)

In [27]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')


geo_df.head()

# google python merge two files

# https://codescracker.com/python/program/python-program-merge-two-files.htm

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
merged_inner = pd.merge(left=df,right=geo_df, left_on='Postcode', right_on='Postal Code')
merged_inner.shape
dfmerged = merged_inner.drop("Postal Code", axis=1)
dfmerged = dfmerged.rename(columns={"Postcode": "PostalCode"})
dfmerged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4P,Central Toronto,Davisville North,43.712751,-79.390197
1,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
2,M3L,North York,Downsview West,43.739015,-79.506944
3,M1V,Scarborough,"Steeles East,Milliken,L'Amoreaux East,Agincour...",43.815252,-79.284577
4,M3C,North York,"Don Mills South,Flemingdon Park",43.725900,-79.340923


In [22]:
Downtown_Toronto_data = dfmerged[dfmerged['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5T,Downtown Toronto,"Kensington Market,Chinatown,Grange Park",43.653206,-79.400049
1,M4X,Downtown Toronto,"St. James Town,Cabbagetown",43.667967,-79.367675
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
3,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636


In [23]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# create map of Manhattan using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'], Downtown_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto